In [1]:
!nvidia-smi

Thu Nov 26 01:40:34 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN RTX           Off  | 00000000:04:00.0 Off |                  N/A |
| 41%   45C    P0    71W / 280W |     11MiB / 24220MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN RTX           Off  | 00000000:05:00.0 Off |                  N/A |
| 53%   63C    P8    32W / 280W |     11MiB / 24220MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [2]:
######### IMPORTING NECESSARY MODULES #########
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
import sys
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

sys.path.append('/home/ironman/abhishek/AliProducts/architectures/')
sys.path.append('/home/ironman/abhishek/AliProducts/Helper/')

from dataloader import mydataset, create_prime_dict 
from trainer import train, test_classify, eval_classify
from Load_model import load



from vit_pytorch import ViT

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


**Dataloading Scheme**

In [4]:
trainlist = '/home/ironman/abhishek/AliProducts/Newlist/NEW_train_list1.txt'
validlist = '/home/ironman/abhishek/AliProducts/Newlist/NEW_valid_list1.txt'

In [5]:
prime_dict = create_prime_dict(trainlist)

Number of classes =  50030


In [6]:
#### Train Dataloader #### 
train_dataset = mydataset(trainlist, prime_dict, name='train')          
train_dataloader = data.DataLoader(train_dataset, shuffle= True, batch_size = 2048, num_workers=16,pin_memory=True)


#### Validation Dataloader #### 
validation_dataset = mydataset(validlist, prime_dict, name='valid')         
validation_dataloader = data.DataLoader(validation_dataset, shuffle=False, batch_size = 256, num_workers=16,pin_memory=True)

**Model Definition**

In [7]:
model = ViT(
    image_size = 256,
    patch_size = 32,
    num_classes = 50030,
    dim = 1024,
    depth = 6,
    heads = 8,
    mlp_dim = 2048,
    dropout = 0.1,
    emb_dropout = 0.1
)


model = nn.DataParallel(model,device_ids=[0,1,2,3]).to(device)
model

# preds = v(img, mask = mask) # (1, 1000)

DataParallel(
  (module): ViT(
    (patch_to_embedding): Linear(in_features=3072, out_features=1024, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (transformer): Transformer(
      (layers): ModuleList(
        (0): ModuleList(
          (0): Residual(
            (fn): PreNorm(
              (norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
              (fn): Attention(
                (to_qkv): Linear(in_features=1024, out_features=3072, bias=False)
                (to_out): Sequential(
                  (0): Linear(in_features=1024, out_features=1024, bias=True)
                  (1): Dropout(p=0.1, inplace=False)
                )
              )
            )
          )
          (1): Residual(
            (fn): PreNorm(
              (norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
              (fn): FeedForward(
                (net): Sequential(
                  (0): Linear(in_features=1024, out_features=2048, bias=True)
            

**Hyperparameters**

In [8]:
## Loss Function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-4, momentum=0.9)


# Epochs
num_Epochs = 120

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 30, gamma = 0.1)

#Cutmix
# beta = 1
# cutmix_prob = 1

In [9]:
modelname = 'ViT'
modelpath = '/home/ironman/abhishek/saved_model_checkpoints/AliProducts/' + modelname

In [10]:
writer = SummaryWriter(modelname)

train(model, train_dataloader, validation_dataloader, criterion, optimizer, lr_scheduler, modelpath, writer, device, epochs = num_Epochs)

writer.flush()
writer.close()

loss 10.822515144348145
loss 21.597200241088867
loss 32.23590029716492
loss 42.71700489997864
loss 52.991988554000855
Epoch:  1
training loss =  10.549524871565454
Validation Loss: 10.0993	Top 1 Validation Accuracy: 0.0006	 Top 5 Validation Accuracy: 0.0021
loss 9.9066588306427
loss 19.672726259231567
loss 38.660869722366336
loss 47.85664821624756
Epoch:  2
training loss =  9.517297350245414
Validation Loss: 9.0108	Top 1 Validation Accuracy: 0.0045	 Top 5 Validation Accuracy: 0.0159
loss 8.733815441131592
loss 17.328718528747558
loss 25.772779321670534
loss 34.03513367652893
loss 42.118423409461975
Epoch:  3
training loss =  8.376714614655475
Validation Loss: 7.8984	Top 1 Validation Accuracy: 0.0244	 Top 5 Validation Accuracy: 0.0687
loss 7.53102397441864
loss 14.948071246147157
loss 22.23593794822693
loss 29.378154759407042
loss 36.36968928813934
Epoch:  4
training loss =  7.237522326785026
Validation Loss: 6.9290	Top 1 Validation Accuracy: 0.0680	 Top 5 Validation Accuracy: 0.1555
lo

loss 0.6680206391215324
loss 1.0040175610780715
loss 1.3411113026738166
loss 1.6800301298499107
Epoch:  33
training loss =  0.33647634608925675
Validation Loss: 4.5044	Top 1 Validation Accuracy: 0.4088	 Top 5 Validation Accuracy: 0.5856
loss 0.30627839744091034
loss 0.612277209609747
loss 0.9245934142172336
loss 1.237934525758028
loss 1.5506626169383526
Epoch:  34
training loss =  0.3106356358035005
Validation Loss: 4.4921	Top 1 Validation Accuracy: 0.4105	 Top 5 Validation Accuracy: 0.5868
loss 0.28877111107110975
loss 0.5790776017308236
loss 0.86720927760005
loss 1.16431432813406
loss 1.4614444233477115
Epoch:  35
training loss =  0.29208808652252605
Validation Loss: 4.5115	Top 1 Validation Accuracy: 0.4111	 Top 5 Validation Accuracy: 0.5871
loss 0.273321919888258
loss 0.5496798089146614
loss 0.8253349187970161
loss 1.1069909557700157
loss 1.3861771747469902
Epoch:  36
training loss =  0.2778615283665897
Validation Loss: 4.5078	Top 1 Validation Accuracy: 0.4112	 Top 5 Validation Accu

Validation Loss: 4.3566	Top 1 Validation Accuracy: 0.4200	 Top 5 Validation Accuracy: 0.5957
loss 0.3103577707707882
loss 0.4672184452414513
loss 0.6241054435819388
loss 0.7790831629186868
Epoch:  65
training loss =  0.15591441382982318
Validation Loss: 4.3528	Top 1 Validation Accuracy: 0.4205	 Top 5 Validation Accuracy: 0.5961
loss 0.15593709394335747
loss 0.3116811175644398
loss 0.46599105320870876
loss 0.62258535631001
loss 0.7783322159945965
Epoch:  66
training loss =  0.15589404090059747
Validation Loss: 4.3487	Top 1 Validation Accuracy: 0.4208	 Top 5 Validation Accuracy: 0.5966
loss 0.1555816727131605
loss 0.311201825812459
loss 0.4666702013462782
loss 0.6206740867346525
loss 0.7764394406229258
Epoch:  67
training loss =  0.15551955861260566
Validation Loss: 4.3479	Top 1 Validation Accuracy: 0.4211	 Top 5 Validation Accuracy: 0.5965
loss 0.15232408449053764
loss 0.30744159646332264
loss 0.4601302193850279
loss 0.6129221346229314
loss 0.7668746693432331
Epoch:  68
training loss = 

KeyboardInterrupt: 

In [ ]:
#### Load saved model from checkpoint  #####
model, optimizer, lr_scheduler, train_loss, v_loss, v_acc, epoch = load(modelpath, model, optimizer, lr_scheduler)


In [12]:
v_loss, top1_acc, accuracy_dict= eval_classify(model, validation_dataloader, criterion, device)
print('Validation Loss: {:.4f}\tTop 1 Validation Accuracy: {:.4f}\nAccuracy:{}\t'.format(v_loss, top1_acc, accuracy_dict))

Validation Loss: 0.6577	Top 1 Validation Accuracy: 0.8654
Accuracy:defaultdict(<class 'int'>, {'Top 1 Accuracy': 86.54392178672833, 'Top 5 Accuracy': 96.05128355238122, 'Top 10 Accuracy': 97.26604806369464, 'Top 20 Accuracy': 98.00954248748647, 'Top 30 Accuracy': 98.32860111816878, 'Top 50 Accuracy': 98.66814975265639, 'Top 100 Accuracy': 99.03404267775078})	
